In [ ]:
import numpy as np
import sklearn as sk
import sklearn.datasets as sk_ds
import sklearn.svm as sk_svm
import matplotlib.pyplot as plt

In [ ]:
wbc_features, wbc_targets = sk_ds.load_breast_cancer(True)
print('WBC: Zbiór {} danych z {} cechami'.format(len(wbc_targets),
                                                 wbc_features.shape[1]))

## SVM

### Klasyfikator liniowy

$$ y = f(\vec{w} \cdot \vec{x}) = f(\sum_i w_i x_i)  $$

$\vec{w}$ - wektor wag klasyfikatora

$\vec{x}$ - wektor wejściowy cech

$f$ - funkcja decyzyjna $\Re \to N $ odwzorowuje zbiór liczb rzeczywistych na klasę 

Dla dwóch klas funkcja $f$ zazwyczaj przypisuje iloczyn skalarny do jednej klasy jeśli jest on większy od pewnej wartości i do drugiej klasy jeśli nie jest. W takim przypadku klasyfikator liniowy dzieli przestrzeń wejściową za pomocą hiperpłaszczyzny, czyli uogólnienia płaszczyzny do n-wymiarów.

Dla takiego przypadku hiperpłaszczyzna w klasyfikatorze liniowym może być opisana jako zbiór puntów $\vec x$ dla których zachodzi $ \vec w \cdot \vec x - b = 0 $.

In [ ]:
def plot_svm_2d(ax, data, labels, classifier, plot_hyperplanes=False):
  min_x = data[:,0].min() - 1
  max_x = data[:,0].max() + 1
  min_y = data[:,1].min() - 1
  max_y = data[:,1].max() + 1
  
  xx, yy = np.meshgrid(np.arange(min_x, max_x, 0.01),
                       np.arange(min_y, max_y, 0.01))
  Z = classifier.predict(np.c_[xx.ravel(), yy.ravel()])
  Z = Z.reshape(xx.shape)
  ax.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.8)
  ax.scatter(data[:,0], data[:,1], c=labels, cmap=plt.cm.coolwarm)
  
  # Hyperplane
  if plot_hyperplanes:
    w = classifier.coef_[0]
    a = -w[0] / w[1]
    xx2 = np.linspace(min_x, max_x)
    yy2 = a * xx2 - (classifier.intercept_[0]) / w[1]
  
    margin = 1 / np.sqrt(np.sum(classifier.coef_ ** 2))
    yy_down = yy2 - np.sqrt(1 + a ** 2) * margin
    yy_up = yy2 + np.sqrt(1 + a ** 2) * margin


    ax.plot(xx2, yy2, 'k-')
    ax.plot(xx2, yy_down, 'k--')
    ax.plot(xx2, yy_up, 'k--')
  
  ax.set_xlim(min_x, max_x)
  ax.set_ylim(min_y, max_y)

def make_linear(fig, plot_hyperplanes=False,
                C=1.0, kernel='linear', **kwargs):
  in_features = wbc_features[40:80,:2]
  out_targets = wbc_targets[40:80]
  
  classifier = sk_svm.SVC(kernel=kernel, C=C, **kwargs)
  classifier.fit(in_features, out_targets)
  plot_svm_2d(fig, in_features, out_targets, classifier, plot_hyperplanes)
  return classifier.score(in_features, out_targets)

  


fig = plt.figure()
ax = fig.add_subplot(1,1,1)
score = make_linear(ax)
print(score)
plt.show()

### Klasyfikator SVM

SVM (ang. Support Vector Machine) jest to rodzaj klasyfikatora liniowego. Polega on na wyznaczeniu hiperpłaszczyzny tak, aby margines pomiędzy hiperpłaszczyzną i najbliższymi punktami (wektorami nośnymi - support vectors) był jak największy.

W tym celu definiujemy dwie dodatkowe hiperpłaszczyzny, równoległe do wspomnianej poprzednio, o równaniach:

$$
\vec w \cdot \vec x - b = 1
$$
$$
\vec w \cdot \vec x - b = -1
$$

Odpowiadają one hiperpłaszczyznom granicznym, opartym na wektorach nośnych.

Odległość między tym płaszczyznami to $\frac{2}{\|\vec w \|}$, naszym zadaniem jest więc minimaliacja

Minimalizuje $\|\vec{w}\|$ tak aby zachodziło równanie:

$$y_i (\vec{w} \cdot \vec{x}_i - b) \ge 1$$

, gdzie 

$\vec{w}$ - wektor wag klasyfikatora, 

$b$ - hiperpłaszczyzna, 

$\vec{x}_i$ - punkty zbioru treningowego, 

$y_i$ - klasa $\{-1, 1\}$ do której należy punkt $x_i$

Równanie to działa tylko dla danych liniowo separowalnych, dla zbioru nieseparowalnego konieczne jest uogólnienie sformułowania:

Zminimalizuj 
$$\| w \|^2 + C \sum_i \xi_i $$
tak aby zachodziło 
$$y_i(\vec w \cdot \vec x_i - b) \ge 1 - \xi_i,\ \xi_i \ge 0,\ i=1,...,n$$

, gdzie

$\xi_i$ - współczynnik błędu dla danego punktu (*"jak daleko wchodzi do złej klasy"*)

$C$ - współczynnik określający stosunek między zwiększeniem marginesu, a minimalizacją błędów

### Postać dualna


Takie sformułowanie oznacza problem znalezienia ekstremum warunkowego. W celu efektywnego rozwiązania tego problemu przy pomocy metod numerycznych stosuje się metodę mnożników Lagrange'a.
W tym celu problem przekształcony jest do postaci dualnej:

Zakładając, że 
$$\vec w = \sum_i \alpha_i y_i \vec x_i$$
zmaksymalizuj 
$$
W(\vec \alpha)=\sum_i \alpha_i - \frac 1 2 \sum_i \sum_j \alpha_i \alpha_j y_i y_j (\vec x_i \cdot \vec x_j)
$$
, tak aby zachodziło $\sum_i \alpha_i y_i = 0$ oraz $0\le \alpha_i \le C $ dla $i = 1,..,n$.

Jedynymi wektorami dla których $\alpha_i \ne 0$ są wektory nośne.

In [ ]:
fig = plt.figure(figsize=(6.4*3, 4.8))
Cs = [0.005, 0.05, 0.5, 1.0]
for i, c in enumerate(Cs):
  ax = fig.add_subplot(1, len(Cs), i + 1)
  accuracy = make_linear(ax, True, C=c)
  ax.set_title("C = {:2.2f}, acc = {:2.2f}".format(c, accuracy))
plt.show()

### Trik kernelowy

Klasyfkator liniowy, a więc i SVM może poprawnie klasyfikować tylko dane liniowo separowalne, co stanowi bardzo wąską klasę problemów. W tym celu stosuje się **trik kernelowy** oparty na idei, że dane nieseparowalne w *n*-wymiarach mogą być liniowo separowalne w wyższych wymiarach, *n*+1, *n*+2, itd.

W tym celu definiujemy nieliniowe przekształcenie punktów wejściowych $\phi(\vec x_i)$ oraz funkcję kernela $k(\vec x_i, \vec x_j)$ spełniającą zależność $k(\vec x_i,\vec x_j ) = \phi(\vec x_i) \cdot \phi(\vec x_j)$.

Na przykład transformacja dodająca dodatkową współrzędną zawierającą odległość od środka układu współrzędnych:
$$
\phi(x): \Re_2 \to \Re_3 
$$
$$
\phi((x_1, x_2)) = (x_1, x_2, \sqrt {x_1^2 + x_2^2})
$$

In [ ]:
def phi(X):
  result = np.zeros((X.shape[0], X.shape[1] + 1))
  result[:, :2] = X
  result[:, 2] = np.sqrt(X[:, 0]**2 + X[:, 1]**2)
  return result

def circle_kernel(X, X_fit):
  return np.dot(phi(X), phi(X_fit).T)
  

def circle():
  from mpl_toolkits.mplot3d import Axes3D
  
  coordinates = np.random.randn(40, 2)
  labels = np.zeros((40,))
  labels[coordinates[:,0]**2 + coordinates[:,1]**2 <= 1.2] = 1
  
  classifier = classifier = sk_svm.SVC(kernel=circle_kernel)
  classifier.fit(coordinates, labels)
  
  figure = plt.figure(figsize=(6.4*2*1.5, 4.8*1.5))
  ax = figure.add_subplot(1, 2, 1)
  
  plot_svm_2d(ax, coordinates, labels, classifier)
  ax.set_title('$\phi((x_1, x_2)) = (x_1, x_2, \sqrt{x_1^2 + x_2^2})$')
  ax = figure.add_subplot(1, 2, 2, projection='3d')
  projected = phi(coordinates)
  ax.scatter(projected[:, 0], projected[:,1], projected[:,2], c=labels,
             cmap=plt.cm.coolwarm, depthshade=False)
  ax.view_init(15, 30)
  
circle()
plt.show()
  

Podstawiając przekształcenie danych wejściowych do równań dla postaci dualnej, otrzymujemy:

Zakładając, że 
$$\vec w = \sum_i \alpha_i y_i \phi(\vec x_i)$$
zmaksymalizuj 
$$
W(\vec \alpha)=\sum_i \alpha_i - \frac 1 2 \sum_i \sum_j \alpha_i \alpha_j y_i y_j k(\vec x_i, \vec x_j)
$$
, tak aby zachodziło $\sum_i \alpha_i y_i = 0$ oraz $0\le \alpha_i \le C $ dla $i = 1,..,n$.

Najczęściej stosowanymi kernelami są:
- wielomianowy $k(\vec x_i, \vec x_j) = (\vec x_i \cdot \vec x_j + c)^d$
- Gaussowski $k(\vec x_i, \vec x_j) = \exp(-\frac{\|\vec x_i - \vec x_j \|^2}{2 \sigma^2})$
- sigmoidalny $k(\vec x_i, \vec x_j) = \tanh(\kappa(\vec x_i \cdot \vec x_j) + \Theta)$

In [ ]:
fig = plt.figure(figsize=(6.4*3, 4.8))
kernels = ['linear', 'poly', 'sigmoid', 'rbf']
kwargs = [dict(), dict(), dict(), dict()]
kwargs[2]['gamma'] = 0.0008
for i, kernel in enumerate(kernels):
  ax = fig.add_subplot(1, len(Cs), i + 1)
  accuracy = make_linear(ax, kernel=kernel, **kwargs[i])
  ax.set_title("kernel = '{}', acc = {:2.4f}".format(kernel, accuracy))
plt.show()

### Jakość klasyfikacji

#### ROC - Receiver Operating Characteristic 
Krzywa ROC służy do oceny jakości klasyfikatora. Przedstawia zależność ilości prawdziwie pozytywnych ($T_p$) i fałszywie pozytywnych wyników ($T_f$). Krzywa idealnego klasyfikatora obejmuje lewy górny punkt wykresu (100% prawdziwie pozytywnych wyników i 0% fałszywie pozytywnych), zatem zazwyczaj większe pole pod wykresem jest lepsze. Istotne jest także nachylenie wykresu, bardziej stromy jest lepszy.

In [ ]:
from sklearn import datasets
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

bc = datasets.load_breast_cancer()
X = bc.data
y = bc.target

indices = y < 2
X_train, X_test, y_train, y_test = train_test_split(X[indices], y[indices], test_size=0.5, random_state=2)

classifier = KNeighborsClassifier(n_neighbors=3)
classifier.fit(X_train, y_train)

probs = classifier.predict_proba(X_test)
# keep probabilities for the positive outcome only
probs = probs[:, 1]

area_under_curve = roc_auc_score(y_test, probs)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, probs)

plt.plot([0, 1], [0, 1], linestyle='--', label='no skill')
plt.plot(false_positive_rate, true_positive_rate, marker='.', label='ROC curve (area = {0:.2f})'.format(area_under_curve))

plt.legend(loc="lower right")
plt.xlabel('T_f (false positive rate)')
plt.ylabel('T_p (true positive rate)')
plt.show()

#### Precision-recall
Miara precision-recall służy do określania jakości klasyfikatora, szczególnie jeżeli liczebność obu klas znacznie się różni. Składa się z dwóch parametrów:
* **precision** ($P$): stosunek liczby prawdziwie pozytywnych wskazań ($T_p$) do liczby prawdziwie pozytywnych i fałszywie pozytywnych wskazań klasyfikatora ($T_p + T_f$)
$$ P = \frac{T_p}{T_p+T_f}$$
* **recall** ($R$): stosunek liczby prawdziwie pozytywnych wskazań ($T_p$) do liczby prawdziwie pozytywnych i fałszywie negatywnych wskazań klasyfikatora ($T_p + F_n$)
$$ R = \frac{T_p}{T_p+F_n}$$
Obie wartości zależą od progu klasyfikatora.

Zmniejszanie progu może zwiększyć ilość zwróconych wyników ($T_p+T_f$). Jeżeli próg był ustawiony zbyt wysoko, to dodatkowe wyniki będą prawdziwie pozytywne, dzięki czemu zwiększy się precision i recall. Jeżeli próg był odpowiedni lub zbyt niski, to w dodatkowych wynikach będzie dużo fałszywie pozytywnych punktów, przez co spadnie precision, a recall niewiele się zwiększy. Obrazuje to kształt wykresu:

In [ ]:
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.metrics import average_precision_score
import matplotlib.pyplot as plt

bc = datasets.load_breast_cancer()
X = bc.data
y = bc.target
# split into train/test sets, selecting only 2 classes
indices = y < 2
X_train, X_test, y_train, y_test = train_test_split(X[indices], y[indices], test_size=0.5, random_state=2)


classifier = KNeighborsClassifier(n_neighbors=3)
classifier.fit(X_train, y_train)
# predict probabilities
probs = classifier.predict_proba(X_test)
# keep probabilities for the positive outcome only
probs = probs[:, 1]
# predict class values
yhat = classifier.predict(X_test)
precision, recall, thresholds = precision_recall_curve(y_test, probs)
f1 = f1_score(y_test, yhat)
area_under_curve = auc(recall, precision)
average_precision = average_precision_score(y_test, probs)

print('f1=%.3f auc=%.3f ap=%.3f' % (f1, area_under_curve, average_precision))

plt.plot([0, 1], [0.5, 0.5], linestyle='--', label='no skill')
plt.plot(recall, precision, marker='.', label='precision-recall curve (area = {0:.2f})'.format(area_under_curve))

plt.legend(loc="best")
plt.xlabel('R (recall)')
plt.ylabel('P (precision)')
plt.show()

#### F1
F1 to miara jakości klasyfikatora. Jest średnią harmoniczną wartości precision i recall. Najlepszy klasyfikator ma F1 równy 1, a najgorszy 0.
$$F1 = (\frac{P^{-1} + R^{-1}}{2})^{-1} = 2\frac{P\cdot R}{P+R}$$

#### Różnica między krzywymi ROC i precision-recall


In [ ]:
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.metrics import average_precision_score
import matplotlib.pyplot as pyplot
# generate 2 class dataset
X, y = make_classification(n_samples=1000, n_classes=2, weights=[0.9,0.1], random_state=1)
# split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=2)
# fit a model
model = KNeighborsClassifier(n_neighbors=3)
model.fit(X_train, y_train)
# predict probabilities
probs = model.predict_proba(X_test)
# keep probabilities for the positive outcome only
probs = probs[:, 1]
# calculate AUC
roc_auc = roc_auc_score(y_test, probs)
print('area under ROC curve: %.3f' % roc_auc)
# calculate roc curve
false_positive_ratio, true_positive_ratio, thresholds = roc_curve(y_test, probs)
plt.plot([0, 1], [0, 1], linestyle='--', label='no skill')
plt.plot(false_positive_ratio, true_positive_ratio, marker='.', label='ROC curve')
plt.legend(loc='best')
plt.xlabel('T_f (false positive rate)')
plt.ylabel('T_p (true positive rate)')
plt.show()

# predict class values
yhat = model.predict(X_test)

precision, recall, thresholds = precision_recall_curve(y_test, probs)
f1 = f1_score(y_test, yhat)
pr_auc = auc(recall, precision)
ap = average_precision_score(y_test, probs)
print('f1=%.3f auc=%.3f ap=%.3f' % (f1, pr_auc, ap))

plt.plot([0, 1], [0.1, 0.1], linestyle='--', label='no skill')
plt.plot(recall, precision, marker='.', label='precision-recall curve (area = {0:.2f})'.format(area_under_curve))
plt.legend(loc="lower right")
plt.xlabel('R (recall)')
plt.ylabel('P (precision)')
plt.show()

## Działanie SVM

* Dane ze zbioru WBC dzielimy na K podzbiorów
* Dla każdego podzbioru trenujemy SVM, pozostałe podzbiory służą do weryfikacji
* Obliczamy parametry działania dla każdego z klasyfikatorów z osobna i łączne, dla czałego zbioru danych

In [ ]:
def plot_all_folds(X, y, k_fold, classifier, title):
    print(title)
    f, axes = plt.subplots(1, 2, figsize=(10, 5))

    y_real = []
    y_proba = []
    
    for i, (train_index, test_index) in enumerate(k_fold.split(X, y)):
        # split dataset
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # train classifier
        classifier.fit(X_train, y_train)
        # predict probabilities
        proba = classifier.predict_proba(X_test)
        proba = proba[:,1]
        
        # calculate and plot ROC curve
        roc_auc = roc_auc_score(y_test, proba)
        false_positive_ratio, true_positive_ratio, thresholds = roc_curve(y_test, proba)
        roc_label = 'Fold {}: ROC AUC={:.4f}'.format(i+1, roc_auc)
        axes[0].step(false_positive_ratio, true_positive_ratio, marker='.', label=roc_label)
        
        # calculate and plot precision-recall curve, F1 score and average precision
        yhat = classifier.predict(X_test)
        precision, recall, _ = precision_recall_curve(y_test, proba)
        f1 = f1_score(y_test, yhat)
        pr_auc = auc(recall, precision)
        ap = average_precision_score(y_test, proba)

        pr_label = 'Fold {}: PR AUC={:.4f}'.format(i+1, auc(recall, precision))
        axes[1].step(recall, precision, marker='.', label=pr_label)
    
        print('Fold {}: ROC auc={:.4f} f1={:.4f} P-R auc={:.4f} ap={:.4f}'.format(i, roc_auc, f1, pr_auc, ap))
        
        y_real.append(y_test)
        y_proba.append(proba)

    # contatenate values from whole dataset
    y_real = np.concatenate(y_real)
    y_proba = np.concatenate(y_proba)
    
    # calculate average ROC curve and plot reference value
    false_positive_ratio, true_positive_ratio, _ = roc_curve(y_real, y_proba)
    roc_label = 'Overall AUC={:.4f} {:.4f}'.format(auc(true_positive_ratio, false_positive_ratio),roc_auc_score(y_real, y_proba) )
    axes[0].step(false_positive_ratio, true_positive_ratio, label=roc_label, lw=2, color='black')
    axes[0].plot([0, 1], [0, 1], linestyle='--', label='no skill')
    
    # calculate average precision-recall curve and plot reference value
    precision, recall, _ = precision_recall_curve(y_real, y_proba)
    pr_label = 'Overall AUC=%.4f' % (auc(recall, precision))
    axes[1].step(recall, precision, label=pr_label, lw=2, color='black')
    axes[1].plot([0, 1], [0.5, 0.5], linestyle='--', label='no skill')
    
    axes[0].legend(loc='best', fontsize='small')
    axes[0].set_xlabel('T_f (false positive rate)')
    axes[0].set_ylabel('T_p (true positive rate)')
    axes[1].set_xlabel('R (recall)')
    axes[1].set_ylabel('P (precision)')
    axes[1].legend(loc='best', fontsize='small')

    f.tight_layout()
    plt.show()

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import cross_validate
import numpy as np
FOLDS = 5
SAMPLES = 200

X = wbc_features[:SAMPLES]
y = wbc_targets[:SAMPLES]

# prepare classifiers
classifiers = [
    sk_svm.SVC(gamma='auto', kernel='linear', C=1.0, probability=True, random_state=1),
    sk_svm.SVC(gamma=0.001, kernel='rbf', probability=True, random_state=1),
    sk_svm.SVC(gamma='auto', kernel='rbf', probability=True, random_state=1),
    sk_svm.SVC(gamma='auto', kernel='poly', C=1.0, probability=True, random_state=1),
]

k_fold = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=1)

for classifier in classifiers:
    title = 'SVC kernel: {}'.format(classifier.kernel)
    plot_all_folds(X, y, k_fold, classifier, title)

## Wizualizacja wyników za pomocą PCA

In [ ]:
from sklearn import decomposition
from sklearn import datasets
from sklearn import preprocessing


X = wbc_features[:SAMPLES]
y = wbc_targets[:SAMPLES]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=2)

pca = decomposition.PCA()

# aplikacja PCA na zbiorze danych X
pca.fit(X)
X_transformed = pca.transform(X)



for classifier in classifiers:

    classifier.fit(X, y)
    # plot predictions
    plt.scatter(X_transformed[y_pred==0, 0], X_transformed[y_pred==0, 1], color='none', edgecolor='darkred', s=50)
    plt.scatter(X_transformed[y_pred==1, 0], X_transformed[y_pred==1, 1], color='none', edgecolor='darkblue', s=50)
    
    # plot expected values
    plt.scatter(X_transformed[y==0, 0], X_transformed[y==0, 1], color='red', s=20)
    plt.scatter(X_transformed[y==1, 0], X_transformed[y==1, 1], color='blue', s=20)
    plt.title(classifier.kernel)
    plt.show()

## Zadanie



In [ ]:
import pandas

def get_values(final_type=np.int):
  """
  Get arrhythmia database.
  """
  
  # 13th column contains too much missing data to be usefull
  columns = list(range(13))+list(range(14, 280))

  arrhythmia = pandas.read_csv(
      'http://archive.ics.uci.edu/ml/machine-learning-databases/' \
      'arrhythmia/arrhythmia.data', header=None, usecols=columns)
  arrhythmia = arrhythmia.values

  # For other missing values we generate values with Gaussian distribution 
  # with mean and standard deviation of existing values
  def calculate_mean(column):
    return column[column != '?'].astype(np.float32).mean()

  def calculate_std(column):
    return column[column != '?'].astype(np.float32).std()

  missing_columns = set()
  for i, row in enumerate(arrhythmia):
    for j, value in enumerate(row):
      if value == '?':
        missing_columns.add(j)

  for col in missing_columns:
    mean = calculate_mean(arrhythmia[:, col])
    std = calculate_std(arrhythmia[:, col])
    for j in range(arrhythmia.shape[0]):
      if arrhythmia[j, col] == '?':
        arrhythmia[j, col] = np.random.normal(mean, std)

  features = arrhythmia[:,:278].astype(final_type)
  # We substitute one to get nicer {0,1} classes for binary classification
  targets = arrhythmia[:,278].astype(np.int) - 1
  
  # Remove classes with too few data
  for i in range(15):
    if np.sum(targets == i) < 5:
      features = features[targets != i,:]
      targets = targets[targets != i]
  return features, targets

features, targets = get_values()
binary_targets = np.copy(targets)
binary_targets[binary_targets > 0] = 1

print('Arrhythmia: Zbiór {} danych z {} cechami'.format(*features.shape))

# Number of cross-validation splits
splits = 5

# Metrics to report
# ==============================================================================
# TODO Add aditional metrics

def accuracy(result, target):
  return np.sum(result == target) / len(target) * 100

metrics = [accuracy]

# ==============================================================================


def test_classifier(builder, features, labels, metrics=metrics, *args, **kwargs):
  """
  builder - function producing the classifier to test
  args and kwargs will be passed to the builder
  
  Returns mean tuple (mean_accuracy, accuracy_deviation)
  """
  
  # Stratified KFold mainains class distribution in iterations
  fold = sk.model_selection.StratifiedKFold(splits)
  
  scores = []
  for train, test in fold.split(features, labels):
    # Build the model
    classifier = builder(*args, **kwargs)
    # Train it
    classifier.fit(features[train], labels[train])
    # Use it to predict test lables
    predicted = classifier.predict(features[test])
    score = [m(predicted, labels[test]) for m in metrics]
    scores.append(score)
  
  # Report the results as mean +/- standard deviation
  scores = np.array(scores)
  means = scores.mean(axis=0)
  stds = scores.std(axis=0)
  for i, metric in enumerate(metrics):
    print('{}: {:2.2f}%(+/-{:2.2f}%)'.format(metric.__name__,
                                             means[i], stds[i]))
  

In [ ]:
# Test linear classifier
def linear_svm_builder(): 
  return sk_svm.SVC(kernel='linear')

print('Simple linear classifier:')
test_classifier(linear_svm_builder, features, targets)

In [ ]:
# TODO Test SVM classifiers for different kernels (poly, sigmoid, etc)
#      and different kernel parameters (degree, gamma, C)
# TODO Compare with k-NN for k=1,3,5
# TODO Apply PCA